In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from matplotlib import pyplot as plt
from xgboost.sklearn import XGBClassifier

In [2]:
df = pd.read_csv("./train.csv")

In [3]:
testdf = pd.read_csv("./test-3.csv")

In [4]:
df.head()

,AC,AF,AR,AS,AST,AY,AwayTeam,Date,FTR,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam,league
0,3.0,12.0,0.0,14.0,4.0,0.0,Stuttgart,07/08/09,H,6.0,12.0,0.0,13.0,7.0,0.0,0.0,0.0,Wolfsburg,bundesliga
1,1.0,10.0,0.0,7.0,0.0,1.0,FC Koln,08/08/09,H,16.0,8.0,0.0,24.0,11.0,0.0,0.0,0.0,Dortmund,bundesliga
2,3.0,20.0,0.0,15.0,3.0,2.0,Hannover,08/08/09,H,5.0,16.0,0.0,10.0,4.0,0.0,0.0,3.0,Hertha,bundesliga
3,10.0,28.0,0.0,9.0,3.0,2.0,Bayern Munich,08/08/09,D,3.0,10.0,0.0,9.0,1.0,1.0,1.0,0.0,Hoffenheim,bundesliga
4,5.0,28.0,0.0,13.0,7.0,2.0,Leverkusen,08/08/09,D,3.0,22.0,0.0,8.0,4.0,2.0,1.0,1.0,Mainz,bundesliga


In [5]:
testdf.head()

,index,AC,AF,AR,AS,AST,AY,AwayTeam,Date,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam,Referee,league
0,0,5,17,0,19,4,2,Leverkusen,18/08/17,4,13,0,13,8,0,2,1,Bayern Munich,NaN,bundesliga
1,1,7,16,0,13,1,3,Augsburg,19/08/17,3,18,0,11,5,0,1,4,Hamburg,NaN,bundesliga
2,2,1,12,0,9,2,1,Stuttgart,19/08/17,5,18,0,10,3,0,0,2,Hertha,NaN,bundesliga
3,3,4,15,0,11,2,3,Werder Bremen,19/08/17,6,17,0,14,3,0,0,1,Hoffenheim,NaN,bundesliga
4,4,1,14,0,6,2,5,Hannover,19/08/17,13,18,0,14,6,0,0,3,Mainz,NaN,bundesliga


In [7]:
testdf.shape

(1826, 20)

In [9]:
testdf_cleaned = testdf

In [8]:
df_cleaned = df.dropna(axis = 0, how ='any') 

In [6]:
import pickle
draw_pred_rfmodel = pickle.load(open('rf_model2.sav', 'rb'))

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
testdf_cleaned["diffHaltimeGoals"] = abs(testdf_cleaned["HTAG"]-testdf_cleaned["HTHG"])
testdf_cleaned["diffShots"] = abs(testdf_cleaned["AS"]-testdf_cleaned["HS"])
testdf_cleaned["diffShotsOnTarget"] =  abs(testdf_cleaned["AST"]-testdf_cleaned["HST"])
testdf_cleaned["diffCorners"] =  abs(testdf_cleaned["AC"]-testdf_cleaned["HC"])

testdf_cleaned["diffFouls"] =  abs(testdf_cleaned["AF"]-testdf_cleaned["HF"])
testdf_cleaned["diffYcards"] =  abs(testdf_cleaned["AY"]-testdf_cleaned["HY"])
testdf_cleaned["diffRCards"] =  abs(testdf_cleaned["AR"]-testdf_cleaned["AR"])

In [14]:
testdf_cleaned['HomeAwayteam']=testdf_cleaned['HomeTeam'] + "-X-"+ testdf_cleaned['AwayTeam']
X= testdf_cleaned
X["AwayTeamEnc"] = le.fit_transform(X["AwayTeam"].astype(str))
X["HomeTeamEnc"] = le.fit_transform(X["HomeTeam"].astype(str))
X["leagueEnc"] = le.fit_transform(X["league"].astype(str))
X["HomeAwayteamEnc"] = le.fit_transform(X["HomeAwayteam"].astype(str))
X= X.drop("AwayTeam", axis=1)
X=X.drop("HomeTeam", axis=1)
X= X.drop("league", axis=1)
X= X.drop("Date", axis=1)
X= X.drop("HomeAwayteam", axis=1)

In [15]:
selectf = ['AR', 'AY', 'HC', 'HR', 'HST', 'HTAG', 'HTHG', 'diffHaltimeGoals',
       'diffShots', 'diffShotsOnTarget']

In [16]:
X1= X[selectf]

In [17]:
predDr = draw_pred_rfmodel.predict(X1)

In [18]:
X["drawpred"] = predDr

In [23]:
X.shape

(1826, 28)

In [21]:
import pickle
xgbmodel = pickle.load(open('xgbmodel.sav', 'rb'))

In [25]:
X= X.drop("index", axis=1)
X= X.drop("Referee", axis=1)

In [26]:
ypred = xgbmodel.predict(X)

In [27]:
ypred

array([2, 2, 1, ..., 0, 2, 2])

In [32]:


le.fit_transform(['A', 'D', 'H'])
print(le.classes_)


matchpreds = le.inverse_transform(ypred)

['A' 'D' 'H']


In [35]:
matchpreds

array(['H', 'H', 'D', ..., 'A', 'H', 'H'], dtype='<U1')

In [41]:
np.array(matchpreds)

array(['H', 'H', 'D', ..., 'A', 'H', 'H'], dtype='<U1')

In [46]:
np.savetxt("matchpreds.txt", np.array(matchpreds), fmt="%s")